In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style> .widget-hslider {  width: 950px; margin-left: -10px;} </style>"))


import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, fixed, Layout
import ipywidgets as widgets
from filters import NanAndCloseTo1Filter,NoFilter
from transformations.peakutils_wrapper import BaselineCorrection,PeakMarker

from statsmodels.tsa.stattools import adfuller
from datasets.datasets import getDataset
from plotter import SingleCellPlotter,SingleConditionPlotter
from transformations.transformations import TransformationPipeline, ClearNans, NormalizedByMax, SavGolFilter, Difference
from transformations.transformations import removeNans
from transformations.tisean_wrapper import TiseanTransformation
from filters import MinConsecutiveLengthFilter

dataset = getDataset("hke3_batch1")
filter_ = MinConsecutiveLengthFilter(40)
tp = TransformationPipeline([])

previosly_done = []

with open("res.txt",'r') as pin:
    for line in pin:
        s_line = line.split()
        previosly_done.append((s_line[0],int(s_line[1])))

previosly_done = set(previosly_done)
        
def getnext():
    tuples_ = []
    for scd in dataset._scd:  # iterate over conditions
        for scmd in scd._scmd:  # iterate over cells
            pos = scmd._position
            idx = scmd._cell_idx
            ts = scmd.get_time_series()
            if filter_.filter(ts, pos, idx):
                continue
            if (pos, idx) in previosly_done:
                continue
            tuples_.append((ts, pos, idx))
    for i,tuple_ in enumerate(tuples_):
        yield tuple_
        
tuples = getnext()
global tuple_
tuple_=tuples.next()



def f(xmin=0,xmax=299):
    global tuple_
    plt.figure(figsize=(10,5))
    plt.plot(tuple_[0],color='blue')
    plt.title(str(tuple_[1])+" "+str(tuple_[2]))
    plt.xlabel('Time [1 Frame = 10 mins]')
    plt.ylabel('FRET/CFP')
    plt.xlim([-1,301])
    plt.axvline(x=xmin,ymin=0,ymax=2,color='red')
    plt.axvline(x=xmax,ymin=0,ymax=2,color='red')
    plt.ylim([0.75,1.15])
    plt.grid()
    ts,ts_idx = removeNans(tuple_[0][int(xmin):int(xmax)])
    pvalue = adfuller(ts, regression='c')[1]
    plt.text(0,0.75,"{0:.3f}".format(pvalue))
    plt.show()
                 

global w
w = interactive(f, xmin=(0.0,299.0,1.0), xmax=(0.0,299.0,1.0))
display(w)

def on_click_done(b):
    global w
    global tuple_
    with open("res.txt",'a') as pout:
        pout.write(tuple_[1]+" "+str(tuple_[2]))
        pout.write(" "+str(int(w.kwargs['xmin']))+" "+str(int(w.kwargs['xmax'])))
        pout.write('\n')
    
    tuple_ = tuples.next()    
    w.close()
    w = interactive(f, xmin=(0.0,299.0,1.0), xmax=(0.0,299.0,1.0))
    display(w)
    display(HTML("<style> .widget-hslider {  width: 950px; margin-left: -10px;} </style>"))
    
def on_click_discard(b):
    global w
    global tuple_
    tuple_ = tuples.next()
    with open("res.txt",'a') as pout:
        pout.write(tuple_[1]+" "+str(tuple_[2]))
        pout.write(" None None")
        pout.write('\n')
    w.close()
    w = interactive(f, xmin=(0.0,299.0,1.0), xmax=(0.0,299.0,1.0))
    
    
    display(w)
    display(HTML("<style> .widget-hslider {  width: 950px; margin-left: -10px;} </style>"))

button = widgets.Button(description='Done!')
display(button)


button_discard = widgets.Button(description='Discard!')
display(button_discard)
    

button_discard.on_click(on_click_discard)
button.on_click(on_click_done)

display(HTML("<style> .widget-hslider {  width: 950px; margin-left: -10px;} </style>"))
